# **Importing Required Packages**

In [2]:
pip install cvlib

In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

# **Preprocessing for Eyes**

In [58]:
#Eye Categories
eyes = {'Closed':0, 'Open':1}
path = '/content/drive/MyDrive/Drowsiness/train/'

In [59]:
#Converting Photos to Numpy Array
photos = []
labels = []
for cat in eyes:
  temp = path+cat+'/'
  for j in os.listdir(path+cat):
    img = cv2.imread(temp+j)
    img = cv2.resize(img,(96,96))
    photos.append(img)
    labels.append(eyes[cat])
np.save('/content/drive/MyDrive/Drowsiness/Eyes_Features.npy',photos)
np.save('/content/drive/MyDrive/Drowsiness/Eyes_Labels.npy',labels)

# **Preprocessing for Face**

In [61]:
#Face Categories
path = '/content/drive/MyDrive/Drowsiness/train/'
face_cat = {'yawn':2,'no_yawn':3}

In [62]:
#Converting Face images to numpy array
photos = []
labels = []
for cat in face_cat:
  temp = path+cat+'/'
  for j in os.listdir(path+cat):
    img = cv2.imread(temp+j)
    face = cv.detect_face(img)
    for k in face[0]:
      x1,y1,x2,y2 = k
      crop = np.copy(img[y1:y2,x1:x2])
      try:
        crop = cv2.resize(crop,(96,96))
        photos.append(crop)
        labels.append(face_cat[cat])
      except:
        break
np.save('/content/drive/MyDrive/Drowsiness/Face_Feature.npy',photos)
np.save('/content/drive/MyDrive/Drowsiness/Face_Labels.npy',labels)
  

# **Loading Data and Preprocessing**

In [4]:
#Loading Face Features and Labels
X_face = np.load('/content/drive/MyDrive/Drowsiness/Face_Feature.npy')
y_face = np.load('/content/drive/MyDrive/Drowsiness/Face_Labels.npy')
#Loading Eyes Features and Labels
X_eye = np.load('/content/drive/MyDrive/Drowsiness/Eyes_Features.npy')
y_eye = np.load('/content/drive/MyDrive/Drowsiness/Eyes_Labels.npy')

In [5]:
#Merging both face and eye features and labels
X = np.concatenate((X_face, X_eye))
y = np.concatenate((y_face, y_eye))

In [8]:
#Splitting Data for test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [9]:
#Features preprocessing
X_train = X_train.reshape(X_train.shape[0], 96, 96, 3).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 96, 96, 3).astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
#Labels Preprocessing
y_train = np_utils.to_categorical(y_train, num_classes=4).astype('float32')
y_test = np_utils.to_categorical(y_test, num_classes=4).astype('float32')

In [11]:
#Function to create augmented images
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

# **Function To Create Base model**

In [20]:
def getmodel():
  model = Sequential()
  #model.add(Conv2D(256, (3,3), activation='relu', input_shape=(96, 96, 3)))
  #model.add(MaxPool2D(3,3))
  #model.add(Dropout(0.2))
  model.add(Conv2D(128, (3,3), activation='relu', input_shape=(96,96,3)))
  model.add(MaxPool2D(3,3))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3,3), activation='relu'))
  model.add(MaxPool2D(3,3))
  model.add(Dropout(0.2))
  model.add(Conv2D(32, (3,3), activation='relu'))
  model.add(MaxPool2D(3,3))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(1000, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  return model

# **Model Creation and Training**

In [21]:
#Creating Model
model = getmodel()

In [22]:
#Model Compiling
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
#Training Model
model.fit(aug.flow(X_train, y_train, batch_size=80),validation_data=(X_test, y_test), epochs=300)
model.save('/content/drive/MyDrive/Drowsiness/model.h5')

Epoch 1/300
29/29 [==============================] - 23s 202ms/step - loss: 1.3643 - accuracy: 0.3031 - val_loss: 0.9880 - val_accuracy: 0.6207
Epoch 2/300
29/29 [==============================] - 5s 176ms/step - loss: 1.0260 - accuracy: 0.5321 - val_loss: 0.6604 - val_accuracy: 0.7759
Epoch 3/300
29/29 [==============================] - 5s 181ms/step - loss: 0.7509 - accuracy: 0.6834 - val_loss: 0.4928 - val_accuracy: 0.8500
Epoch 4/300
29/29 [==============================] - 5s 181ms/step - loss: 0.6500 - accuracy: 0.7285 - val_loss: 0.4256 - val_accuracy: 0.8259
Epoch 5/300
29/29 [==============================] - 5s 179ms/step - loss: 0.5402 - accuracy: 0.7788 - val_loss: 0.4284 - val_accuracy: 0.8569
Epoch 6/300
29/29 [==============================] - 5s 181ms/step - loss: 0.4812 - accuracy: 0.8052 - val_loss: 0.3531 - val_accuracy: 0.8948
Epoch 7/300
29/29 [==============================] - 5s 177ms/step - loss: 0.4127 - accuracy: 0.8212 - val_loss: 0.2688 - val_accuracy: 0.903

# **Model Evaluation**

In [24]:
model.evaluate(X_test, y_test)

19/19 [==============================] - 0s 7ms/step - loss: 0.0747 - accuracy: 0.9690


[0.07470028847455978, 0.9689655303955078]